In [73]:
#required libraries
import numpy as np
import pygame
import tensorflow as tf
import time

In [74]:
# global constants

SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
BG_COLOR = (120, 120, 120)
DRAW_COLOR = (50, 50, 50)

DRAW_RADIUS = 2
ERASE_RADIUS = 5

CAR_WIDTH, CAR_HEIGHT = 20, 40
DEFAULT_START_X, DEFAULT_START_Y = 892, 426
DEFAULT_START_ANGLE = -45
DEFAULT_START_SPEED = 0
ACCELERATION = 0.05
BRAKE_FORCE = 0.1
MAX_SPEED = 5.00
FRICTION = 0.025
MIN_TURN_ANGLE = 1.5
MAX_TURN_ANGLE = 2

TRACK_SAVE_PATH = "track1.png"
CAR_IMAGE_PATH = "Track_images/car.png"
TRACK_IMAGE_PATH = r"track1.png"

FOR making th track drawing 
left ckick for drawing the track
right click for eraing the track

In [75]:
def draw_track():
    """
    Function to draw the racing track.
    Left mouse button to draw, right mouse button to erase.
    Press 'S' to save the track as an image.
    """
    pygame.init()
    pygame.display.set_caption("Drawing the track, press 'S' to save as image.")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    screen.fill(BG_COLOR)
    running, drawing, erase = True, False, False
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    drawing = True
                elif event.button == 3:
                    erase = True
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    drawing = False
                elif event.button == 3:
                    erase = False
            elif event.type == pygame.MOUSEMOTION:
                mouse_pos = pygame.mouse.get_pos()
                if drawing:
                    pygame.draw.circle(screen, DRAW_COLOR, mouse_pos, DRAW_RADIUS)
                elif erase:
                    pygame.draw.circle(screen, BG_COLOR, mouse_pos, ERASE_RADIUS)
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
                pygame.image.save(screen, TRACK_SAVE_PATH)
        pygame.display.update()
    pygame.quit()

In [ ]:
class Car:
    """
    Class representing a car in the racing simulation.
    """
    def __init__(self, image_path, x, y, angle=0, speed=0):
        image = pygame.image.load(image_path).convert_alpha()
        self.image = pygame.transform.scale(image, (CAR_WIDTH, CAR_HEIGHT))
        self.x, self.y = x, y
        self.angle = angle
        self.speed = speed
        self.rect=self.image.get_rect(center=(self.x,self.y))

    def move(self):
        rad = np.radians(self.angle)
        self.x += self.speed * np.cos(rad)
        self.y -= self.speed * np.sin(rad)
        self.rect.center = (self.x, self.y)

    def draw(self, screen):
        rotated_img = pygame.transform.rotate(self.image, self.angle)
        rect = rotated_img.get_rect(center=(self.x, self.y))
        self.mask=pygame.mask.from_surface(rotated_img)
        screen.blit(rotated_img, rect.topleft)
        
    def get_rect(self):
        return self.rect

In [77]:
def ray_casting(car, screen, track_surface):
    sensor_distance = []
    sensor_endpoint = []
    sensor_angle = [-45, 0, 45]

    for angle in sensor_angle:
        ray_angle = car.angle + angle
        ray_x, ray_y = car.x, car.y
        distance = 0
        max_distance = 200

        while distance < max_distance:
            rad = np.radians(ray_angle)
            ray_x += np.cos(rad)
            ray_y -= np.sin(rad)
            distance += 1

            if not (0 <= ray_x < SCREEN_WIDTH and 0 <= ray_y < SCREEN_HEIGHT):
                break

            pixel_color = track_surface.get_at((int(ray_x), int(ray_y)))[0:3]
            if pixel_color == DRAW_COLOR:
                break

        sensor_distance.append(distance)
        sensor_endpoint.append((ray_x, ray_y))

    return sensor_distance, sensor_endpoint

In [ ]:
def run_simulation(
    start_x=DEFAULT_START_X,
    start_y=DEFAULT_START_Y,
    start_angle=DEFAULT_START_ANGLE,
    start_speed=DEFAULT_START_SPEED
):
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Car Simulation")

    track_image = pygame.image.load(TRACK_IMAGE_PATH).convert()
    track_surface = track_image.copy()

    car = Car(CAR_IMAGE_PATH, start_x, start_y, start_angle, start_speed)

    screen.fill(BG_COLOR)
    screen.blit(track_image, (0, 0))

    CAR_ROAD_COLOR = track_surface.get_at((start_x, start_y))[:3]

    finish_line_rect=pygame.Rect(start_x+30,start_y-20,10,100)

    lap_start_time=0
    best_lap_time=float('inf')
    current_lap_time=0
    lap_started=False
    car_on_finish=False

    car_max_speed = 0
    
    #checkpoints
    checkpoints=[]
    checkpoint_data=[
        (834,520,10,120,0),
        (600,540,10,120,0),
        (110,569,10,120,0),
        (285,483,10,120,0),
        (366,314,10,120,0),
        (355,173,10,120,0),
        (450,109,10,120,0),
        (606,170,10,120,0),
        (818,91,10,120,0),
        (1127,88,10,120,0),
        (1094,270,10,120,0),
        (920,346,10,120,0)
    ]
    for x,y,w,h,a in checkpoint_data:
        base_surface=pygame.Surface((w,h),pygame.SRCALPHA)
        base_surface.fill((255,0,255,100))
        rotated_surface=pygame.transform.rotate(base_surface,a)
        rect=rotated_surface.get_rect(center=(x,y))
        mask=pygame.mask.from_surface(rotated_surface)
        checkpoints.append({'surface': rotated_surface, 'rect': rect, 'mask': mask})
    current_checkpoint=0
        

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        keys = pygame.key.get_pressed()
        if car.speed > 0:
            car.speed -= FRICTION
        if car.speed < 0:
            car.speed += FRICTION
        if abs(car.speed) < FRICTION:
            car.speed = 0
        if keys[pygame.K_DOWN]:
            car.speed -= BRAKE_FORCE
        if keys[pygame.K_UP]:
            if car.speed < MAX_SPEED:
                car.speed += ACCELERATION
                
        if car.speed > 0.1:
            speed_factor = car.speed / MAX_SPEED
            turn_range = MAX_TURN_ANGLE - MIN_TURN_ANGLE
            dynamic_turn_angle = MAX_TURN_ANGLE - (speed_factor * turn_range)
        
            if keys[pygame.K_LEFT]:
                car.angle += dynamic_turn_angle
            if keys[pygame.K_RIGHT]:
                car.angle -= dynamic_turn_angle

        screen.blit(track_image, (0, 0))

        font = pygame.font.Font(None, 22)
        text_surface = font.render(f"Speed: {car.speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 50))

        # Ray casting
        distance, sensor_endpoint = ray_casting(car, screen, track_surface)
        text_surface = font.render(f"Distances: {distance}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 80))
        for endpoint in sensor_endpoint:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), endpoint, 1)

        car.move()
        
        #cheking checkpoints
        if current_checkpoint<len(checkpoints):
            target_checkpoint=checkpoints[current_checkpoint]
            offset_x=target_checkpoint['rect'].x-car.rect.x
            offset_y=target_checkpoint['rect'].y-car.rect.y
            if car.mask.overlap(target_checkpoint['mask'],(offset_x,offset_y)):
                current_checkpoint+=1
                print(f"Checkpoint reached!{current_checkpoint}")
        
        car.draw(screen)
        
        #lap timing (collision detection)
        is_colliding=car.get_rect().colliderect(finish_line_rect)
        if is_colliding and not car_on_finish:
            car_on_finish=True
            if not lap_started:                   ##   <<<<  change this start from here <<<<
                lap_started=True   
                lap_start_time=time.time()
            else:
                lap_time=time.time()-lap_start_time
                best_lap_time=min(best_lap_time,lap_time)
                lap_start_time=time.time()
        if not is_colliding:
            car_on_finish=False
            
        pygame.draw.rect(screen, (250, 100, 50), finish_line_rect)
        

        # Collision detection (including out-of-bounds)
        crashed = False
        car_center_pos = (int(car.x), int(car.y))
        if not (0 <= car.x < SCREEN_WIDTH and 0 <= car.y < SCREEN_HEIGHT):
            crashed = True
        else:
            try:
                pixel_color = track_surface.get_at(car_center_pos)[:3]
                if pixel_color != CAR_ROAD_COLOR:
                    crashed = True
            except IndexError:
                crashed = True

        if crashed:
            lap_started=False
            current_lap_time=0 
            text_surface = font.render("Car has crashed!", True, (255, 255, 255))
            screen.blit(text_surface, (50, 110))
            pygame.display.update()
            pygame.time.wait(1000)  # Show message for 1 second
            car.x, car.y = start_x, start_y
            car.angle = start_angle
            car.speed = start_speed

        if car.speed > car_max_speed:
            car_max_speed = car.speed

        if lap_started:
            current_lap_time = time.time() - lap_start_time
            current_lap_str = f"Lap: {current_lap_time:.2f}s"
        else:
            current_lap_str = "Lap: 0.00s"                                           # remove the below condition after adding check points
        best_lap_str = f"Best: {best_lap_time:.2f}s" if best_lap_time != float('inf') and lap_time>30 else "Best: N/A"
        text_surface = font.render(current_lap_str, True, (255, 255, 255))            # remove this ^^^ condition
        screen.blit(text_surface, (50, 500))
        best_text = font.render(best_lap_str, True, (255, 255, 255))
        screen.blit(best_text, (50, 530))
        text_surface = font.render(f"Max speed: {car_max_speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 470))
        
        if event.type == pygame.MOUSEMOTION:
            mouse_point = event.pos
            text_surface = font.render(f"Location: {mouse_point}", True, (255, 255, 255))
            screen.blit(text_surface, (50, 100))

        pygame.display.update()
        clock.tick(60)

    pygame.quit()

        if event.type == pygame.MOUSEMOTION:
            mouse_point = event.pos
            text_surface = font.render(f"Location: {mouse_point}", True, (255, 255, 255))
            screen.blit(text_surface, (50, 100))
 #code for getting mouse location
 


In [ ]:
if __name__ == "__main__":
    #draw_track()
    run_simulation()


AttributeError: 'Car' object has no attribute 'mask'

: 